# FlexML: Flexible ML accelerator, workhorse of TinyVers Heterogeneous Multi-core SoC 
### (Participating in category 1)

```
KU Leuven MICAS Team
SPDX-License-Identifier: Apache-2.0
```

|Name|Affiliation|Email|IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|:----------:|
|Vikram Jain (Leader)|PhD researcher, KU Leuven, Belgium|vikram.jain@kuleuven.be|Yes|Yes|
|Prof. Marian Verhelst (Supervisor)|Professor, KU Leuven, Belgium|marian.verhelst@kuleuven.be|Yes|Yes|

This notebook implements the silicon proven FlexML accelerator of the overall TinyVers Heterogeneous SoC in SKY130 tech using open-source tools.
The original FlexML (part of TinyVers SoC) is prototyped on chip in a commercial 22nm FDSOI tech and published in [VLSI'21](https://ieeexplore.ieee.org/abstract/document/9830409) and [JSSC'23](https://ieeexplore.ieee.org/abstract/document/10022047).

FlexML is a specialized digital ML accelerator of the TinyVers Heterogeneous SoC designed for flexible and energy-efficient deployment of ML workloads on battery and resource-constrained extreme edge devices. Extreme edge ML processing, brings several new challenges: 1.) As these nodes are battery-operated, the system is typically severely power or energy constrained requiring ultra-low power operation, with the ability to idle. 2.) most MCU, moreover, has limited compute power and memory space, resulting in a critical trade-off between model size, execution performance and hardware complexity; 3.) despite the need for efficiency, the system should also be flexible enough to support different classes of NN models across different applications, and 4.) it should have a small footprint. The salient FlexML features to address these challenges and further improve energy efficiency are as follows:
   
 * Hardware-supported, zero-latency runtime dataflow reconfiguration.
 * Precision-scalable MAC units inside each PE to support INT 2/4/8 data types.
 * Zero-skipping support for blockwise structured sparsity and deconvolution operator.
 * Support Vector Machines support with hardware extension in PEs to support subtraction, squaring and rounding.
 * Support for multiple workloads including Convolutional Neural Networks, Fully Connected, Temporal Convolutional Networks, Autoencoders, Generative Adversial Networks, Recurrent Neural Networks, and Support Vector Machines.

These features of FlexML achieve state-of-the-art throughput and energy efficiency over other ML accelerators and SoCs for ML on one of the most diverse workloads. When combined with the TinyVers SoC, the flexibility is extended with the RISC-V and power management providing extensive support for non-ML as well as ML workloads at the extreme edge.

**_Index Terms:_** ML accelerator, flexible, open source, end-to-end flow, multi-modal AI, heterogeneous multi-core, extreme edge ML, OpenLANE, OpenRAM

Citations: 

Jain, V., Giraldo, S., De Roose, J., Boons, B., Mei, L., Verhelst, M. (2022). TinyVers: A 0.8-17 TOPS/W, 1.7 μW-20 mW, Tiny Versatile System-on-chip with State-Retentive eMRAM for Machine Learning Inference at the Extreme Edge. In: 2022 IEEE Symposium on VLSI Technology and Circuits (VLSI Technology and Circuits). Presented at the 2022 IEEE Symposium on VLSI Technology and Circuits (VLSI Technology and Circuits), Honolulu, HI, USA. ISBN: 78-1-6654-9772-5. doi:10.1109/VLSITechnologyandCir46769.2022.9830409

Jain, V., Giraldo, S., Roose, J.D., Mei, L., Boons, B., Verhelst, M. (2023). "TinyVers: A Tiny Versatile System-on-Chip With State-Retentive eMRAM for ML Inference at the Extreme Edge." Ieee Journal Of Solid-State Circuits, 1-12. doi: 10.1109/JSSC.2023.3236566


## FlexML (TinyVers) architecture

The following figure shows the TinyVers and the FlexML architecture designed for flexible energy-efficient ML workloads at the extreme edge. The figure also shows the dataflow reconfiguration, the internals of the processing element, the support for zero-skipping in structured sparsity and deconvolution. This notebook implements the FlexML with 16 (4x4) processing elements with reduced private L1 memories.

The complete TinyVers RTL is available at https://github.com/vikramjain236/tinyvers

This notebook has been inspired by https://github.com/sscs-ose/sscs-ose-code-a-chip.github.io/tree/main/ISSCC23/accepted_notebooks/DAG_Processing_Unit

![FlexML and TinyVers architecture](./figs/flexml_tinyvers_arch.png)

## Design flow
This notebook performs the physical implementation of DPU in SKY130 using only open-source CAD tool:
*   The OpenLANE script flow which uses:
    *   Yosys for logic synthesis
    *   Openroad for placing and routing
    *   Klayout to produce the final GDS file
    *   Magic for DRC, LVS and PEX.
*   OpenRAM to generate SRAM macros

Set up a conda enviornment and install SKY130 PDK, OpenLANE, and other required tools.

In [2]:
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
import os
import pathlib
import sys

conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}

!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        open_pdks.sky130a \
                        magic \
                        openroad \
                        netgen \
                        yosys \
                        openlane
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel conda-forge \
                        tcllib gdstk pyyaml click pandas svgutils pip

bin/micromamba
env: CONDA_PREFIX=/esat/rhea1/users/vjain/openhw_examples/conda-env
env: PATH=/esat/rhea1/users/vjain/openhw_examples/conda-env/bin:/users/micas/vjain/anaconda3/envs/openhw/bin:/users/micas/vjain/anaconda3/condabin:/usr/lib64/qt-3.3/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/puppetlabs/bin:/sbin:/software/bin:/users/micas/micas/bin:/esat/rhea1/users/vjain/klayout/usr/bin
env: LD_LIBRARY_PATH=/esat/rhea1/users/vjain/openhw_examples/conda-env/lib::/esat/rhea1/users/vjain/klayout/usr/lib64/klayout

                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/

Empty environment created at prefix: /esat/rhea1/users/vjain/openhw_examples/conda-env

                                           __
          __  ______ ___  ____ _____ ___  

litex-hub/linux-64 ââââââââââââââââââââââââ 837.7kB @ 593.1kB/s Downloaded  1.7slitex-hub/linux-64                                 @ 593.1kB/s  1.8s

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /esat/rhea1/users/vjain/openhw_examples/conda-env

  Updating specs:

   - open_pdks.sky130a
   - magic
   - openroad
   - netgen
   - yosys
   - openlane


  Package                              Version  Build                 Channel                  Size
âââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââââ
  Install:
ââââââââââââââââââââââââââââââââââââââââââââââââââââââââââ

Linking open_pdks.sky130a-1.0.404_0_gbb4b48f-20230225_164303
Linking _libgcc_mutex-0.1-main
Linking libstdcxx-ng-11.2.0-h1234567_1
Linking ca-certificates-2023.01.10-h06a4308_0
Linking ld_impl_linux-64-2.38-h1181459_1
Linking libgomp-11.2.0-h1234567_1
Linking _openmp_mutex-5.1-1_gnu
Linking libgcc-ng-11.2.0-h1234567_1
Linking libuuid-1.41.5-h5eee18b_0
Linking lz4-c-1.9.4-h6a678d5_0
Linking gmp-6.2.1-h295c915_3
Linking bzip2-1.0.8-h7b6447c_0
Linking yaml-0.2.5-h7b6447c_0
Linking libffi-3.4.2-h6a678d5_6
Linking pixman-0.40.0-h7f8727e_1
Linking xz-5.2.10-h5eee18b_1
Linking libxcb-1.15-h7f8727e_0
Linking jpeg-9e-h5eee18b_1
Linking icu-58.2-he6710b0_3
Linking expat-2.4.9-h6a678d5_0
Linking ncurses-6.4-h6a678d5_0
Linking openssl-1.1.1t-h7f8727e_0
Linking pcre-8.45-h295c915_0
Linking fmt-8.1.1-hd09550d_1
Linking zlib-1.2.13-h5eee18b_0
Linking readline-8.2-h5eee18b_0
Linking spdlog-1.9.2-hd09550d_0
Linking zstd-1.5.4-hc292b87_0
Linking libpng-1.6.39-h5eee18b_0
Linking glib-2.69.1-he621ea3_2
Li

conda-forge/noarch   ââââââ¸ââââââââââââââââ¸â   7.2MB /  ??.?MB @   4.1MB/s  1.8s[+] 1.9s
conda-forge/linux-64 âââââââââ¸ââââââââââââââ   7.8MB /  ??.?MB @   4.2MB/s  1.9s
conda-forge/noarch   âââââââââ¸ââââââââââââââ   7.7MB /  ??.?MB @   4.2MB/s  1.9s[+] 2.0s
conda-forge/linux-64 ââââââââââââ¸âââââââââââ   8.1MB /  ??.?MB @   4.1MB/s  2.0s
conda-forge/noarch   âââââââââââ¸ââââââââââââ   7.9MB /  ??.?MB @   4.0MB/s  2.0s[+] 2.1s
conda-forge/linux-64 ââââââââââââââ¸âââââââââ   8.5MB /  ??.?MB @   4.1MB/s  2.1s
conda-forge/noarch   âââââââââââââ¸ââââââââââ   8.3MB /  ??.?MB @   4.0MB/s  2.1s[+] 2.2s
conda-forge/linux-64 ââââââââââââââââ¸ââââ

conda-forge/noarch   ââââââââââââââââââââââ  11.8MB @   4.0MB/s Finalizing  4.4s[+] 4.5s
conda-forge/linux-64 âââââââââââ¸âââââââââââ  12.8MB @   4.3MB/s             4.5s
conda-forge/noarch   ââââââââââââââââââââââ  11.8MB @   4.0MB/s Finalizing  4.5s[+] 4.6s
conda-forge/linux-64 âââââââââââ¸âââââââââââ  12.8MB @   4.3MB/s             4.6s
conda-forge/noarch   ââââââââââââââââââââââ  11.8MB @   4.0MB/s Finalizing  4.6s[+] 4.7s
conda-forge/linux-64 âââââââââââ¸âââââââââââ  12.8MB @   4.3MB/s             4.7s
conda-forge/noarch   ââââââââââââââââââââââ  11.8MB @   4.0MB/s Finalizing  4.7s[+] 4.8s
conda-forge/linux-64 âââââââââââ¸âââââââââââ  12.8

conda-forge/linux-64 âââââââââ¸ââââââââââââââ  17.0MB /  ??.?MB @   2.2MB/s  7.8s[+] 7.9s
conda-forge/linux-64 âââââââââ¸ââââââââââââââ  17.0MB /  ??.?MB @   2.2MB/s  7.9s[+] 8.0s
conda-forge/linux-64 âââââââââ¸ââââââââââââââ  17.0MB /  ??.?MB @   2.2MB/s  8.0s[+] 8.1s
conda-forge/linux-64 âââââââââ¸ââââââââââââââ  17.0MB /  ??.?MB @   2.2MB/s  8.1s[+] 8.2s
conda-forge/linux-64 ââââââââââ¸âââââââââââââ  17.0MB /  ??.?MB @   2.1MB/s  8.2s[+] 8.3s
conda-forge/linux-64 ââââââââââââ¸âââââââââââ  17.0MB /  ??.?MB @   2.1MB/s  8.3s[+] 8.4s
conda-forge/linux-64 ââââââââââââ¸âââââââââââ  17.0MB /  ??.?MB @   2.1MB/s  8.4s[+] 8.5s
conda-forge/linux-64 ââââââââââââ

conda-forge/linux-64 âââââââââ¸ââââââââââââââ  27.8MB /  ??.?MB @   2.2MB/s 12.5s[+] 12.6s
conda-forge/linux-64 ââââââââââ¸âââââââââââââ  28.2MB /  ??.?MB @   2.2MB/s 12.6s[+] 12.7s
conda-forge/linux-64 ââââââââââââ¸âââââââââââ  28.6MB /  ??.?MB @   2.3MB/s 12.7s[+] 12.8s
conda-forge/linux-64 ââââââââ¸âââââââââââââââ  28.9MB /  ??.?MB @   2.3MB/s 12.8s[+] 12.9s
conda-forge/linux-64 ââââââââââ¸âââââââââââââ  29.3MB /  ??.?MB @   2.3MB/s 12.9s[+] 13.0s
conda-forge/linux-64 ââââââââââââ¸âââââââââââ  29.5MB /  ??.?MB @   2.3MB/s 13.0s[+] 13.1s
conda-forge/linux-64 âââââââââââââ¸ââââââââââ  29.6MB /  ??.?MB @   2.3MB/s 13.1s[+] 13.2s
conda-forge/linux-64 âââââââââ

conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 17.8s[+] 17.9s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 17.9s[+] 18.0s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 18.0s[+] 18.1s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 18.1s[+] 18.2s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 18.2s[+] 18.3s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 18.3s[+] 18.4s
conda-forge/linux-64 ââââââââââââââââââââââ  30.6MB @   2.3MB/s Finalizing 18.4s[+] 18.5s
conda-forge/linux-64 ââââââââââââââ

### OpenRAM compiler
Download the OpenRAM compiler and install the prerequisites.

In [27]:
!git clone -b v1.1.19 https://github.com/VLSIDA/OpenRAM.git
!which pip3
!pip3 install -r OpenRAM/requirements.txt

Cloning into 'OpenRAM'...
remote: Enumerating objects: 37166, done.
remote: Counting objects: 100% (10770/10770), done.
remote: Compressing objects: 100% (2142/2142), done.
remote: Total 37166 (delta 8593), reused 10656 (delta 8507), pack-reused 26396
Receiving objects: 100% (37166/37166), 55.52 MiB | 22.04 MiB/s, done.
Resolving deltas: 100% (29967/29967), done.
Note: checking out 'f66aac3264598eeae31225c62b6a4af52412d407'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b new_branch_name

/esat/rhea1/users/vjain/openhw_examples/conda-env/bin/pip3
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using 

Set up the enviornment for OpenRAM and OpenLANE. 

In [28]:
import os
import pathlib
import sys

openram_root_path = pathlib.Path('OpenRAM')
OPENRAM_ROOT= str(openram_root_path.resolve())
OPENLANE_ROOT=CONDA_PREFIX + '/share/openlane'
PATH=os.environ['PATH']
%env OPENRAM_ROOT={OPENRAM_ROOT}
%env OPENRAM_HOME={OPENRAM_ROOT}/compiler
%env OPENRAM_TECH={OPENRAM_ROOT}/technology/sky130
%env PYTHONPATH={OPENRAM_ROOT}/compiler:{OPENRAM_ROOT}/technology:{OPENRAM_ROOT}/technology/sky130/modules
%env PDK_ROOT={CONDA_PREFIX}/share/pdk
%env PDK=sky130A
%env STD_CELL_LIBRARY=sky130_fd_sc_hd
%env STD_CELL_LIBRARY_OPT=sky130_fd_sc_hd
%env TCLLIBPATH={CONDA_PREFIX}/lib/tcllib1.20
%env OPENLANE_ROOT={OPENLANE_ROOT}
%env PATH={PATH}:{OPENLANE_ROOT}:{OPENLANE_ROOT}/scripts
%env OPENLANE_LOCAL_INSTALL=1
#%env SRAM_LIBRARY = {OPENRAM_ROOT}/sky130_fd_bd_reram
!make -C OpenRAM SRAM_GIT_REPO=https://github.com/google/skywater-pdk-libs-sky130_fd_bd_sram.git
#!make -C OpenRAM SRAM_GIT_REPO=https://github.com/google/skywater-pdk-libs-sky130_fd_pr_reram.git


env: OPENRAM_ROOT=/esat/rhea1/users/vjain/openhw_examples/OpenRAM
env: OPENRAM_HOME=/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler
env: OPENRAM_TECH=/esat/rhea1/users/vjain/openhw_examples/OpenRAM/technology/sky130
env: PYTHONPATH=/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler:/esat/rhea1/users/vjain/openhw_examples/OpenRAM/technology:/esat/rhea1/users/vjain/openhw_examples/OpenRAM/technology/sky130/modules
env: PDK_ROOT=/esat/rhea1/users/vjain/openhw_examples/conda-env/share/pdk
env: PDK=sky130A
env: STD_CELL_LIBRARY=sky130_fd_sc_hd
env: STD_CELL_LIBRARY_OPT=sky130_fd_sc_hd
env: TCLLIBPATH=/esat/rhea1/users/vjain/openhw_examples/conda-env/lib/tcllib1.20
env: OPENLANE_ROOT=/esat/rhea1/users/vjain/openhw_examples/conda-env/share/openlane
env: PATH=/esat/rhea1/users/vjain/openhw_examples/conda-env/bin:/users/micas/vjain/anaconda3/envs/openhw/bin:/users/micas/vjain/anaconda3/condabin:/usr/lib64/qt-3.3/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/s

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_base/sky130_fd_bd_sram__sram_dp_blkinv_base.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_blkinv_base.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_mcon/sky130_fd_bd_sram__sram_dp_blkinv_mcon.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_blkinv_mcon.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_met23/sky130_fd_bd_sram__sram_dp_blkinv_met23.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_blkinv_met23.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_opt1/sky130_fd_bd_sram__sram_dp_blkinv_opt1.gds' -> '/esat/rhea1/users/vjain/openhw_examples/Ope

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt2a/sky130_fd_bd_sram__sram_dp_cell_opt2a.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_cell_opt2a.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt4/sky130_fd_bd_sram__sram_dp_cell_opt4.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_cell_opt4.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt4a/sky130_fd_bd_sram__sram_dp_cell_opt4a.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_cell_opt4a.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt5/sky130_fd_bd_sram__sram_dp_cell_opt5.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../te

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_opt5/sky130_fd_bd_sram__sram_dp_colend_opt5.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_colend_opt5.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_opt5a/sky130_fd_bd_sram__sram_dp_colend_opt5a.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_colend_opt5a.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_opt6/sky130_fd_bd_sram__sram_dp_colend_opt6.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_colend_opt6.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_opt6a/sky130_fd_bd_sram__sram_dp_colend_opt6a.gds' -> '/esat/rhea1/users/vjain/openhw_examples/O

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_rowend_strp_cont/sky130_fd_bd_sram__sram_dp_rowend_strp_cont.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_rowend_strp_cont.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_rowenda/sky130_fd_bd_sram__sram_dp_rowenda.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_rowenda.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_rowendai/sky130_fd_bd_sram__sram_dp_rowendai.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_rowendai.gds'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_rowendb/sky130_fd_bd_sram__sram_dp_rowendb.gds' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../

...

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_horstrap_half2/sky130_fd_bd_sram__sram_dp_horstrap_half2.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_horstrap_half2.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_horstrap_half5/sky130_fd_bd_sram__sram_dp_horstrap_half5.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_horstrap_half5.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_horstrap_li/sky130_fd_bd_sram__sram_dp_horstrap_li.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_horstrap_li.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_horstrap_limcon/sky130_fd_bd_sram__sram_dp_horstrap_limcon.maglef' -> 

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt1a/sky130_fd_bd_sram__sram_dp_swldrv_opt1a.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_opt1a.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt1ai/sky130_fd_bd_sram__sram_dp_swldrv_opt1ai.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_opt1ai.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt1b/sky130_fd_bd_sram__sram_dp_swldrv_opt1b.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_opt1b.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt1bi/sky130_fd_bd_sram__sram_dp_swldrv_opt1bi.maglef' -> '/esat/rhe

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt4di/sky130_fd_bd_sram__sram_dp_swldrv_opt4di.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_opt4di.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_p1lic/sky130_fd_bd_sram__sram_dp_swldrv_p1lic.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_p1lic.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_p1m_ser/sky130_fd_bd_sram__sram_dp_swldrv_p1m_ser.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_p1m_ser.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_p1m_siz/sky130_fd_bd_sram__sram_dp_swldrv_p1m_siz.maglef' -> '/

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_opt1_ce/sky130_fd_bd_sram__sram_sp_cell_opt1_ce.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_cell_opt1_ce.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_opt1a/sky130_fd_bd_sram__sram_sp_cell_opt1a.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_cell_opt1a.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_p1_serif/sky130_fd_bd_sram__sram_sp_cell_p1_serif.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_cell_p1_serif.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_p1_serifs/sky130_fd_bd_sram__sram_sp_cell_p1_serifs.maglef' -> '/esat/rhea1/u

'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_rowend_ce/sky130_fd_bd_sram__sram_sp_rowend_ce.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_rowend_ce.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_rowend_met2/sky130_fd_bd_sram__sram_sp_rowend_met2.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_rowend_met2.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_rowend_p1m_siz/sky130_fd_bd_sram__sram_sp_rowend_p1m_siz.maglef' -> '/esat/rhea1/users/vjain/openhw_examples/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_rowend_p1m_siz.mag'
'/esat/rhea1/users/vjain/openhw_examples/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_rowenda/sky130_fd_bd_sram__sram_sp_rowenda.maglef' -> '/esat/rhea1/users/vjain/openhw

### SRAM specification
Write configuration files with the required SRAM specification. This will be consumed by the OpenRAM compiler.

In [13]:
%%writefile ram_config_512x32.py
"""
Single ported RAM
"""
word_size = 32 # Bits
num_words = 512
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)
write_size = word_size # Bits

# Single port
num_rw_ports = 1
num_r_ports = 0
num_w_ports = 0
num_spare_rows= 1 # required only in 1rw case
num_spare_cols= 1 # requires only for 1rw case
ports_human = '1rw'

tech_name = "sky130"
nominal_corner_only = True

route_supplies = "ring"
uniquify = True

output_name = f"sky130_sram_{ports_human}_{word_size}x{num_words}_{write_size}"
output_path = "./OpenRAM"

Overwriting ram_config_512x32.py


In [14]:
%%writefile ram_config_1024x32.py
"""
Single ported RAM
"""
word_size = 32 # Bits
num_words = 1024
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)
write_size = word_size # Bits

# Single port
num_rw_ports = 1
num_r_ports = 0
num_w_ports = 0
num_spare_rows= 1 # required only in 1rw case
num_spare_cols= 1 # requires only for 1rw case
ports_human = '1rw'

tech_name = "sky130"
nominal_corner_only = True

route_supplies = "ring"
uniquify = True

output_name = f"sky130_sram_{ports_human}_{word_size}x{num_words}_{write_size}"
output_path = "./OpenRAM"

Overwriting ram_config_1024x32.py


In [15]:
%%writefile ram_config_2048x32.py
"""
Single ported RAM
"""
word_size = 32 # Bits
num_words = 2048
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)
write_size = word_size # Bits

# Single port
num_rw_ports = 1
num_r_ports = 0
num_w_ports = 0
num_spare_rows= 1 # required only in 1rw case
num_spare_cols= 1 # requires only for 1rw case
ports_human = '1rw'

tech_name = "sky130"
nominal_corner_only = True

route_supplies = "ring"
uniquify = True

output_name = f"sky130_sram_{ports_human}_{word_size}x{num_words}_{write_size}"
output_path = "./OpenRAM"

Overwriting ram_config_2048x32.py


### SRAM generation
Launch the OpenRAM compiler to generate required macros. This can take up to 2 hours.

In [16]:
!which python3
!python3 $OPENRAM_HOME/openram.py ram_config_512x32.py
!python3 $OPENRAM_HOME/openram.py ram_config_1024x32.py
!python3 $OPENRAM_HOME/openram.py ram_config_2048x32.py


/esat/rhea1/users/vjain/openhw_examples/conda-env/bin/python3
|==============================================================================|
|=========                      OpenRAM v1.1.19                       =========|
|=========                                                            =========|
|=========               VLSI Design and Automation Lab               =========|
|=========        Computer Science and Engineering Department         =========|
|=========            University of California Santa Cruz             =========|
|=========                                                            =========|
|=========          Usage help: openram-user-group@ucsc.edu           =========|
|=========        Development help: openram-dev-group@ucsc.edu        =========|
|=========          Temp dir: /tmp/openram_vjain_1838_temp/           =========|
|=========                See LICENSE for license info                =========|
|==============================================

Only generating nominal corner timing.
Words per row: 8
Output files are: 
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.lvs
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.sp
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.v
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.lib
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.py
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.html
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.log
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.lef
/esat/rhea1/users/vjain/openhw_examples/./OpenRAM/sky130_sram_1rw_32x2048_32.gds
** Submodules: 61.4 seconds
** Placement: 0.1 seconds
**** Retrieving pins: 0.0 seconds
**** Analyzing pins: 0.0 seconds
**** Finding blockages: 17.5 seconds
**** Converting blockages: 0.6 seconds
**

### SRAM placement
Write a configuration file specifying the exact location and orientation of SRAM macros on the die. The instance names are obtained from the netlist after a syntheis run.

In [14]:
%%writefile macro_placement.cfg
ACTIVATION_MEMORY.ACT_MEM_0.generation_RF_column\[0\].generation_blocks_RF_row\[0\].REGISTER_FILE_0.SRAM_i.I1                       200  200  S
ACTIVATION_MEMORY.ACT_MEM_1.generation_RF_column\[0\].generation_blocks_RF_row\[0\].REGISTER_FILE_0.SRAM_i.I1                       1200 200  S
CONTROL_UNIT.WRAPPER_SPARSITY_MEM.sparsity_mem0.SRAM_i.I1                                                                           2600 200  S
CONTROL_UNIT.WRAPPER_SPARSITY_MEM.sparsity_mem1.SRAM_i.I1                                                                           4000 200  S
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_0.row\[0\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    100  2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_0.row\[1\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    700  2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_0.row\[2\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    1300 2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_0.row\[3\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    1900 2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_1.row\[0\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    2500 2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_1.row\[1\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    3100 2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_1.row\[2\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    3700 2000 N
WRAPPER_WEIGHT_MEMORY.UNIFIED_W_1.row\[3\].BLOCK_i.generation_blocks\[0\].generation_per_column\[0\].SRAM_equivalent_i.SRAM_i.I1    4400 2000 N


Overwriting macro_placement.cfg


### Configuration for OpenLANE
Specify the configuration for the synthesis and PnR flow.

In [15]:
%%writefile config.tcl

set ::env(STD_CELL_LIBRARY) "sky130_fd_sc_hd"

set ::env(DESIGN_NAME) cpu
set ::env(VERILOG_FILES) "\
    ./src/*.v \
    "
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_NET) "clk"
set ::env(CLOCK_PERIOD) 100
set ::env(CLOCK_TREE_SYNTH) 1
set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 5000 2500"

set ::env(DESIGN_IS_CORE) 0      
set ::env(SYNTH_MAX_FANOUT) 30

# SRAM config
set ::env(VDD_NETS) "vccd1"
set ::env(GND_NETS) "vssd1"
set ::env(FP_PDN_MACRO_HOOKS) "\
  ACTIVATION_MEMORY.ACT_MEM_0.*.REGISTER_FILE_0.SRAM_i.I1                       vccd1 vssd1 vccd1 vssd1,\
  ACTIVATION_MEMORY.ACT_MEM_1.*.REGISTER_FILE_0.SRAM_i.I1                       vccd1 vssd1 vccd1 vssd1,\
  CONTROL_UNIT.WRAPPER_SPARSITY_MEM.sparsity_mem0.SRAM_i.I1                                                                           vccd1 vssd1 vccd1 vssd1,\
  CONTROL_UNIT.WRAPPER_SPARSITY_MEM.sparsity_mem1.SRAM_i.I1                                                                           vccd1 vssd1 vccd1 vssd1,\
  WRAPPER_WEIGHT_MEMORY.*.SRAM_equivalent_i.SRAM_i.I1    vccd1 vssd1 vccd1 vssd1"

set ::env(MACRO_PLACEMENT_CFG) macro_placement.cfg
set ::env(EXTRA_LEFS)      [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x2048_32.lef $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x512_32.lef $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x1024_32.lef]
set ::env(EXTRA_GDS_FILES) [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x2048_32.gds $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x512_32.gds $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x1024_32.gds]
set ::env(EXTRA_LIBS)      [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x2048_32_TT_1p8V_25C.lib $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x512_32_TT_1p8V_25C.lib $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x1024_32_TT_1p8V_25C.lib]
set ::env(RUN_KLAYOUT_XOR) false
set ::env(MAGIC_DRC_USE_GDS) false
set ::env(QUIT_ON_MAGIC_DRC) false
set ::env(QUIT_ON_LVS_ERROR) false
        
#set ::env(PL_TARGET_DENSITY) 0.5
#set ::env(FP_CORE_UTIL) 60

set ::env(ROUTING_CORES) 8
#set ::env(FP_PDN_HORIZONTAL_HALO) 6
#set ::env(FP_PDN_VERTICAL_HALO) 6
set ::env(DIODE_INSERTION_STRATEGY) 3

# disable klayout because of https://github.com/hdl/conda-eda/issues/175
set ::env(RUN_KLAYOUT) 0
# disable CVC because of https://github.com/hdl/conda-eda/issues/174
set ::env(RUN_CVC) 0

Overwriting config.tcl


### Launch OpenLANE
The entire flow can take up to 5 hours.

In [4]:
!flow.tcl -design . -ignore_mismatches -tag flexml3

OpenLane 2023.03.01_0_ge10820ec-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.tcl'...
[INFO]: PDK Root: /esat/rhea1/users/vjain/openhw_examples/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /esat/rhea1/users/vjain/openhw_examples/runs/flexml3
[INFO]: Removing existing /esat/rhea1/users/vjain/openhw_examples/runs/flexml3...
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[STEP 1]
[INFO]: Running Synthesis (log: runs/flexml3/logs/synthesis/1-synthesis.log)...
[STEP 2]
[INFO]: Running Single-Corner Static Timing Analysis (log: runs/flexml3/logs/synthesis/2-sta.log)...
[STEP 3]
[INFO]: Running

### Output GDS
The output GDS is available in the run directory at runs/flexml3/results/final/gds/. The FlexML GDS can be viewed in an open source layout editor (like klayout) or commercial tool like calibre. Some screenshots from klayout:

#### Full GDS:
![Full GDS](figs/flexml_full_gds.png "Full GDS")

#### GDS without DECAP and FILLER cells
![GDS without DECAP and FILLER cells](figs/flexml_full_gds_wo_fill.png "GDS without DECAP and FILLER cells")

| | |
|:------------:|:-----------:|
|Total die area|5.0 x 2.5 = 12.5 mm<sup>2</sup>|
|Total on-chip SRAM|40 KB|
|Target clock frequency|10 MHz|
|Area of std. cells and SRAM| 4.8 mm<sup>2</sup>|
|Die utilization|38 %|

The detailed report from the tool can be seen in the directory /runs/flexml3/reports/metrics.csv